# Adding support for a new models

# NOTE: This notebook is now out of date and needs to be rewritten to account for ChatTemplates.


Different models are tuned with different role prompt formats. If the model you are using is not already a subclass of `guidance.Model`, you can define your own new subclass with whatever role prompt format you want. Then you can use the guidance role tags and they will get translated into the correct prompt format.

## Orca Mini Chat Example

As an example of how this works, below we implement the [Orca Mini 3B](https://huggingface.co/pankajmathur/orca_mini_3b) prompt. The prompt looks like this:

`### System:\n{system}\n\n### User:\n{instruction}\n\n### Response:\n"`

Where `system` is the system prompt and `instruction` is the user input instructions.

### Define a new OrcaChat class

In [1]:
from guidance import models

# this is our new chat model that inherits from the TransformersChat model and redefines role starts and ends
class OrcaChat(models.transformers.TransformersChat):
    def get_role_start(self, role_name, **kwargs):
        if role_name == "system":
            return "### System:\n"
        elif role_name == "user":
            if str(self).endswith("\n\n### User:\n"):
                return "" # we don't need to start anything if we are starting with a top level unnested system tag
            else:
                return "### System:\n"
        else:
            return " "

    def get_role_end(self, role_name=None):
        if role_name == "system":
            return "\n\n### User:\n"
        elif role_name == "user":
            return "\n\n### Response:\n"
        else:
            return " "

### Load the model

In [2]:
import torch

orca = OrcaChat('pankajmathur/orca_mini_3b', torch_dtype=torch.float16, device_map='auto')
# orca = OrcaChat('gpt2', device_map='auto') # Can use a small mock model while iterating on the prompt

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Check output without indentation formatting

When you designing the prompt format you will want to see exactly how roles are going into the mode without any pretty indentation. Setting `indent=False` allows you to do this (and so is useful during role format development).

In [3]:
from guidance import gen, system, user, assistant, indent_roles

with indent_roles(False):
    with system():
        lm = orca + "You are a cat expert."

    with user():
        lm += "What are the smallest cats?"

    with assistant():
        lm += gen("answer", stop=".", max_tokens=20)

Note that you can also just print the string representation of the model for a truely raw view of the model's context:

In [4]:
print(str(lm))

### System:
You are a cat expert.

### User:
What are the smallest cats?

### Response:
 The smallest cats are the dwarf cats, which include the following breeds:

1 


You can also change just a single role tag:

In [5]:
with indent_roles(False), system():
    lm = orca + "You are a cat expert."

with user():
    lm += "What are the smallest cats?"

with assistant():
    lm += gen("answer", stop=".", max_tokens=20)

### Normal use

When you are satisfied with the correctness of your role formatting you can then use the model like normal:

In [6]:
with system():
    lm = orca + "You are a cat expert."

with user():
    lm += "What are the smallest cats?"

with assistant():
    lm += gen("answer", stop=".", max_tokens=20)

<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>